<a href="https://colab.research.google.com/github/Jasonang-13/ESG-Rating-Gap-Scanner-Using-AI/blob/main/%5BOpen_AI%5D_AI_Scanner_for_MSCI_ESG_Ratings_May_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔑 Step 0: Set your OpenAI API key securely
import os
from getpass import getpass

# Prompt user to enter API key securely
os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

#____________________________________________________________________________

# 📥 Step 1: Upload and Extract Text from PDF ESG Report
from google.colab import files
print("📄 Please upload the Sustainability or Annual Report (PDF format only)")
pdf_uploaded = files.upload()

# Install PyPDF2 quietly
!pip install PyPDF2 --quiet
from PyPDF2 import PdfReader

# Extract text from uploaded PDF
pdf_filename = next((f for f in pdf_uploaded if f.endswith(".pdf")), None)
if pdf_filename:
    reader = PdfReader(pdf_filename)
    text = "".join(page.extract_text() or "" for page in reader.pages)
    print("✔️ PDF ESG report loaded and text extracted.")
else:
    print("❌ No PDF file found. Please upload a valid PDF file.")

#____________________________________________________________________________

# 📊 Step 2: Upload and Load ESG Criteria from Excel
print("\n📊 Please upload the ESG Criteria checklist (Excel format only).")
excel_uploaded = files.upload()

import pandas as pd

excel_filename = next((f for f in excel_uploaded if f.endswith((".xlsx", ".xls"))), None)
if excel_filename:
    df = pd.read_excel(excel_filename)
    esg_criteria = df.iloc[:, 0].dropna().tolist()
    print("✔️ ESG criteria successfully loaded:")
    for i, criterion in enumerate(esg_criteria, 1):
        print(f"{i}. {criterion}")
else:
    print("❌ No Excel file found. Please upload a valid .xlsx or .xls file.")

#____________________________________________________________________________

# 🤖 Step 3: Analyze ESG Report Using OpenAI SDK v1.0.0+
# Install OpenAI SDK if not already installed
!pip install --upgrade openai --quiet

from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Define analysis function
def scan_for_gaps(text, criteria_list):
    prompt = f"""
You are an ESG analyst. Review the following ESG report content and assess the level of disclosure for each criterion listed.

Report Content (excerpt):
{text[:3000]}

Criteria:
{', '.join(criteria_list)}

For each criterion, respond with:
- Disclosure Status: Fully Disclosed / Partially Disclosed / Not Disclosed
- Suggested Improvement (if any)

Respond in a simple bullet list format.
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an ESG analyst."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Run analysis if data is available
if 'text' in globals() and 'esg_criteria' in globals() and text and esg_criteria:
    print("\n📋 ESG Disclosure Analysis:\n")
    print(scan_for_gaps(text, esg_criteria))
else:
    print("⚠️ Skipping analysis: Missing report text or ESG criteria.")